In [1]:
import os
import cv2
import numpy as np
import pickle
import tkinter as tk
from tkinter import simpledialog, messagebox
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC

def load_data(data_path="dataset"):
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    labels, faces_data = [], []
    
    for person_name in os.listdir(data_path):
        person_folder = os.path.join(data_path, person_name)
        if not os.path.isdir(person_folder):
            continue
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue
            faces = face_detector.detectMultiScale(image, 1.1, 5, minSize=(50, 50))
            if len(faces) == 0:
                continue
            x, y, w, h = faces[0]
            face_resized = cv2.resize(image[y:y+h, x:x+w], (100, 100))
            faces_data.append(face_resized.flatten())
            labels.append(person_name)
    
    return np.array(faces_data), np.array(labels)

def train_model():
    faces_data, labels = load_data()
    if len(set(labels)) < 2:
        messagebox.showerror("Error", "At least two different people are required for training.")
        return
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    scaler = StandardScaler()
    faces_data = scaler.fit_transform(faces_data)
    X_train, X_test, y_train, y_test = train_test_split(faces_data, labels, test_size=0.2, random_state=42)
    model = SVC(kernel="linear", probability=True)
    model.fit(X_train, y_train)
    pickle.dump(model, open("face_recognition_model.pkl", "wb"))
    pickle.dump(le, open("label_encoder.pkl", "wb"))
    pickle.dump(scaler, open("scaler.pkl", "wb"))
    messagebox.showinfo("Success", "Model trained successfully!")

def recognize():
    cap = cv2.VideoCapture(0)
    model = pickle.load(open("face_recognition_model.pkl", "rb"))
    le = pickle.load(open("label_encoder.pkl", "rb"))
    scaler = pickle.load(open("scaler.pkl", "rb"))
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.1, 5, minSize=(50, 50))
        
        for (x, y, w, h) in faces:
            face_resized = cv2.resize(gray[y:y+h, x:x+w], (100, 100)).flatten().reshape(1, -1)
            face_resized = scaler.transform(face_resized)
            probabilities = model.predict_proba(face_resized)
            best_class_index = np.argmax(probabilities)
            confidence = probabilities[0][best_class_index]
            predicted_name = le.inverse_transform([best_class_index])[0] if confidence > 0.6 else "Unknown"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, f"{predicted_name} ({confidence*100:.1f}%)", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        cv2.imshow("Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    cap.release()
    cv2.destroyAllWindows()

def add_new_face():
    name = simpledialog.askstring("Input", "Enter new person's name:")
    if not name:
        return
    save_folder = os.path.join("dataset", name)
    os.makedirs(save_folder, exist_ok=True)
    cap = cv2.VideoCapture(0)
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    count = 0
    
    while count < 50:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.1, 5, minSize=(50, 50))
        for (x, y, w, h) in faces:
            face_resized = cv2.resize(gray[y:y+h, x:x+w], (100, 100))
            cv2.imwrite(os.path.join(save_folder, f"{count}.jpg"), face_resized)
            count += 1
        cv2.imshow("Capturing Face", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Success", f"Face dataset for {name} saved successfully!")

def main():
    root = tk.Tk()
    root.title("Face Recognition System")
    tk.Label(root, text="Face Recognition System", font=("Arial", 16)).pack(pady=10)
    tk.Button(root, text="Train Model", command=train_model, width=20).pack(pady=5)
    tk.Button(root, text="Recognize Faces", command=recognize, width=20).pack(pady=5)
    tk.Button(root, text="Add New Person", command=add_new_face, width=20).pack(pady=5)
    tk.Button(root, text="Exit", command=root.quit, width=20).pack(pady=5)
    root.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
import os
import cv2
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tkinter import simpledialog, messagebox
import tkinter as tk

IMG_SIZE = 100  # Image size for CNN
BATCH_SIZE = 32
EPOCHS = 10  # Increase for better training

# Load and preprocess dataset
def load_data(data_path="dataset"):
    labels, faces_data = [], []
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    for person_name in os.listdir(data_path):
        person_folder = os.path.join(data_path, person_name)
        if not os.path.isdir(person_folder):
            continue
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            image = cv2.imread(image_path)
            if image is None:
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
            faces = face_detector.detectMultiScale(image, 1.1, 5, minSize=(50, 50))
            if len(faces) == 0:
                continue
            x, y, w, h = faces[0]
            face_resized = cv2.resize(image[y:y+h, x:x+w], (IMG_SIZE, IMG_SIZE))
            faces_data.append(face_resized)
            labels.append(person_name)

    return np.array(faces_data), np.array(labels)

# Build the CNN model using VGG16
def build_model(num_classes):
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    for layer in base_model.layers[:-4]:
        layer.trainable = False  # Freeze VGG16 layers

    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.8),  # Dropout layer to prevent overfitting
        Dense(num_classes, activation="softmax")  # Output layer
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# Train and save the CNN model
def train_model():
    faces_data, labels = load_data()
    if len(set(labels)) < 2:
        messagebox.showerror("Error", "At least two different people are required for training.")
        return

    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # Normalize images
    faces_data = faces_data / 255.0

    X_train, X_test, y_train, y_test = train_test_split(faces_data, labels, test_size=0.2, random_state=42)

    model = build_model(num_classes=len(set(labels)))
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE)

    model.save("face_recognition_cnn.h5")
    pickle.dump(le, open("label_encoder.pkl", "wb"))

    messagebox.showinfo("Success", "CNN Model trained successfully!")

# Real-time face recognition using CNN
def recognize():
    model = tf.keras.models.load_model("face_recognition_cnn.h5")
    le = pickle.load(open("label_encoder.pkl", "rb"))
    cap = cv2.VideoCapture(0)
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = face_detector.detectMultiScale(gray, 1.2, 6, minSize=(50, 50))

        for (x, y, w, h) in faces:
            face_resized = cv2.resize(frame[y:y+h, x:x+w], (IMG_SIZE, IMG_SIZE))
            face_resized = np.expand_dims(face_resized / 255.0, axis=0)

            predictions = model.predict(face_resized)
            best_class_index = np.argmax(predictions)
            confidence = predictions[0][best_class_index]
            predicted_name = le.inverse_transform([best_class_index])[0] if confidence > 0.6 else "Unknown"

            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(frame, f"{predicted_name} ({confidence*100:.1f}%)", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow("Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

# Add a new face dataset

def add_new_face():
    name = simpledialog.askstring("Input", "Enter new person's name:")
    if not name:
        return
    save_folder = os.path.join("dataset", name)
    os.makedirs(save_folder, exist_ok=True)
    cap = cv2.VideoCapture(0)
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    count = 0
    
    while count < 100:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.1, 5, minSize=(50, 50))
        for (x, y, w, h) in faces:
            face_resized = cv2.resize(frame[y:y+h, x:x+w], (IMG_SIZE, IMG_SIZE))  # Keep it colored (RGB)
            cv2.imwrite(os.path.join(save_folder, f"{count}.jpg"), face_resized)
            count += 1
        cv2.imshow("Capturing Face", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    cap.release()
    cv2.destroyAllWindows()

    # Notify user to retrain the model
    retrain = messagebox.askyesno("Training Required", "New face added! Do you want to retrain the model now?")
    if retrain:
        train_model()
    else:
        messagebox.showinfo("Reminder", "Retrain the model manually before recognition.")

  

# GUI for the application
def main():
    root = tk.Tk()
    root.title("Face Recognition System")
    tk.Label(root, text="Face Recognition System", font=("Arial", 16)).pack(pady=10)
    tk.Button(root, text="Train Model", command=train_model, width=20).pack(pady=5)
    tk.Button(root, text="Recognize Faces", command=recognize, width=20).pack(pady=5)
    tk.Button(root, text="Add New Person", command=add_new_face, width=20).pack(pady=5)
    tk.Button(root, text="Exit", command=root.quit, width=20).pack(pady=5)
    root.mainloop()

if __name__ == "__main__":
    main()


Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 0.2329 - loss: 2.1248 - val_accuracy: 0.3304 - val_loss: 1.9367
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.2432 - loss: 1.8968 - val_accuracy: 0.2087 - val_loss: 1.7805
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.2735 - loss: 1.8731 - val_accuracy: 0.3217 - val_loss: 1.6144
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 86s 6s/step - accuracy: 0.4210 - loss: 1.5497 - val_accuracy: 0.3391 - val_loss: 1.4090
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 86s 6s/step - accuracy: 0.4008 - loss: 1.3674 - val_accuracy: 0.4696 - val_loss: 1.2832
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.5042 - loss: 1.1980 - val_accuracy: 0.4957 - val_loss: 1.3732
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.5992 - loss: 1.0811 - val_accuracy: 0.6435 - val_loss: 1.0184
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.6913 - loss: 0.7953 - val_accuracy: 0.5652 - val_loss:

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 599ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 0.1894 - loss: 2.2927 - val_accuracy: 0.2295 - val_loss: 1.9407
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.1945 - loss: 2.0510 - val_accuracy: 0.1967 - val_loss: 1.9949
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
